*This is a Jupyter Notebook. It is an interactive document that contains both rich text elements such as figures, links, equations, etc. and executable code (in this case Python code) contained in cells.
**How to use a Jupyter Notebook**: You can execute the blocks of code one at the time by placing the mouse in the grey box and pressing shift + enter. An asterisk will appear in the brackets at the top left of the box while the code is being exectued (this may take few seconds) and turns into a number when the execution is over. Alternatively, you can run all the code in the Notebook in a single step by clicking on the menu Cell -> Run All.*

# What are the key controls of a system?
Mathematical models can be used to test hypotheses and increase our understanding of the functioning of the system they represent. Model simulations can be used to explore under which conditions a system will reach a particular state, or to anticipate the effects of human interventions. For example, a mathematical model of an ecological system represents the interactions between different species within a certain habitat. Such a model could be used to identify the conditions under which all the spieces can coexist, or the conditions that would lead to extinction of a particular species; or it could be used to assess the effects of introducing or removing a certain number of individuals of a spieces - for example through hunting or fishing. The underlying assumption here is that the model is a realistic representation of the system, so that what is learnt about the model behaviour applies to the system behaviour too. 
In this Notebook we show how Global Sensitivity Analysis (GSA) can help in this context, and in particular how GSA can help identify the key controls of the system response. 

## An ecological model example
For the sake of illustration, we will use a very simple model representing the dynamics going on between two species. One species, the prey, is the primary source of food source for the other, the predator. The predator-prey food chain persists over time thanks to a **negative feedback loop** *(Ref. 1)*. When predators eat preys, the prey population declines; this reduces the predators' food source and soon leads to the predator population to also decline. When predators decline, the predation rate reduces; this reduces pressure on the preys and soon lead to the prey population to increase again. This cycle creates a fluctuating pattern in the prey and predator population densities which is the long-term (oscillating) equilibrium of the system.

<left><img src="util/predator_prey/predator_prey_equil.gif" width="700px">
(Source: [TED-Ed Youtube video "Feedback loops: How nature gets its rhythms - Anje-Margriet Neutel"]( https://www.youtube.com/watch?v=inVZoI1AkC8))
    
The predator-prey model we use here *(Ref. 2)* is characterised by five parameters:

- *Predator initial population*: the initial number of predator individuals
- *Predator attack rate*: the number of times (per week) that a predator attack is successful, i.e. results in killing and eating a prey
- *Predator efficiency*: the efficiency at turning food into offspring, expressed as the proportion of successful attacks that lead to creating a new predator 
- *Predator death rate*: the fraction of the predator population that dies per week
- *Prey initial population*: initial number of prey individuals

We define as the **optimal habitat equilibrium** of the habitat a situation where both the predator and the prey populations oscillate within the range of **7000 ± 500 individuals each**.

## Reaching the optimal habitat equilibrium
Now imagine we want to determine a combination of the above 5 parameters that would lead the system to reach the optimal habitat equilibrium within a year. This could be useful for example to inform the sustainable management of the habitat, for instance by introducing new individuals (preys or predators) or allowing a certain number of hunting licenses (which is a way to artificially increase the predator death rate). Can you find a combination that put both lines (the predator and prey populations) between the blue dashed lines (7000 ± 500 individuals) before the end of the simulation period?

In [1]:
from ipywidgets import widgets
from util.predator_prey.predator_prey_interactive import predator_prey_interactive

predator_ini,attack_rate,efficiency_rate,death_rate,prey_ini,growth_rate,carrying_capacity,fig1 = predator_prey_interactive()

widgets.VBox([widgets.VBox([predator_ini,attack_rate,efficiency_rate,death_rate,prey_ini],
                           layout=dict(border='solid',width = '750px',height = '180px')),fig1])

The interactive plot above also reports a quantitative indicator of the distance from the equilibrium, called **equilibrium deviation**. This is defined as the absolute difference of the predator population from 7000 individuals (the optimal long-term population value) at the end of the simulation, plus the same absolute difference for the prey population. We would like the deviation to be as close to zero as possible, however finding a set of parameters that achieve a low deviation by varying each of them one-at-a-time is not immediate.

## Finding the key controls of the system response by GSA
The search of the right parameters combination would be easier if we could focus on fewer of them, the ones that mostly control the long-term system response. To identify such parameters, we can apply Global Sensitivity Analysis (GSA). Here we use one particular GSA method, PAWN (Ref. 3). It provides a sensitivity index for each model parameter. The index measures the relative importance of that parameter on the equilibrium deviation: the lower the index, the smaller the influence. The method also provides a threshold value: if the index of a parameter is above the threshold, then the parameter definitely has an effect on the equilibrium deviation, whereas if the index is below the threshold the effect could be negligible. Now run the cell below to obtain the sensivity indices and the threshold (black dashed line).

<left><img src="util/predator_prey/predator_prey_screening_figure.png" width="600px">
    
We observe that two of the parameters, *Predator initial population* and *Prey initial population*, are below the threshold, which means they may have little influence in determining whether the optimal equilibrium is reached or not. Therefore we stop varying these parameters, set them to an arbitrary value and concentrate on the others.
Now focusing on the important parameters *Predator attack rate*, *Predator efficiency ratio* and *Predator death rate*, let's look at their interactions. These can be visually assessed on the coloured scatter plots below. Each point in these plots represents a parameter combination (you can read the parameter values on the horizontal and vertical axes), coloured according to the value of the equilibrium deviation associated to that combination. Blue points represent parameter combinations that reach the optimal equilibrium, i.e. the equlibrium deviation is lower than 1000 individuals, and grey points represent combinations that do not reach the optimal equilibrium.

<left><img src="util/predator_prey/predator_prey_mapping_figures.png" width="700px">
    
The first two plots show that there is only a very specific sub-range of the *Predator attack rate* parameter that can lead to the optimal equilibrium, whereas outside of this narrow range the equilibrium is never achieved, regardless of the value that all the other parameters take. The third plot shows that there is a clear interaction between the *Predator efficiency ratio* and the *Predator death rate*: the optimal equilibrium can be achieved for several different values of these parameters but only if they are both increased/reduced simoultaneously. 

## Reaching the optimal habitat equilibrium, again...
Now taking into account what we have learnt so far about the key controls of the system response, and about the parameter interactions, can you find a combination that reach the **optimal equilibrium** of the predator and prey populations?

In [2]:
predator_ini,attack_rate,efficiency_rate,death_rate,prey_ini,growth_rate,carrying_capacity,fig3 = predator_prey_interactive()

widgets.VBox([widgets.VBox([predator_ini,attack_rate,efficiency_rate,death_rate,prey_ini],
                           layout=dict(border='solid',width = '750px',height = '180px')),fig3])

### References

1. [Feedback loops: How nature gets its rhythms - Anje-Margriet Neutel](https://ed.ted.com/lessons/feedback-loops-how-nature-gets-its-rhythms-anje-margriet-neutel)
2. [Predator-prey model - Hoppensteadt, F. (2006)](http://www.scholarpedia.org/article/Predator-prey_model)
3. [PAWN method - Pianosi and Wagener (2018)](https://doi.org/10.1016/j.envsoft.2018.07.019)